In [2]:
import numpy as np
import math

In [ ]:
def add_zeros(a, numZeros, begin=True, val=0):
    length = a.ravel().shape[0]+numZeros
    out = np.ones(length)*val
    if begin:
        out[:length-numZeros] = a
    else:
        out[numZeros:] = a
    del a
    return out

In [ ]:
filename = "a_example.txt"
with open(filename) as f:
    lines = f.readlines()
    numBooks, numLib, numDays = [int(x) for x in lines[0].split(' ')]
    bookScores = [int(x) for x in lines[1].split(' ')]
    bookScores = np.array(bookScores)
    libs = []
    sequence = []
    for i in range(numLib):
        # numBooksInLib, numDaysReg, booksSendPerDay
        lib = {}
        lib['numBooksInLib'], lib['numDaysReg'], lib['booksSendPerDay'] = tuple([int(x) for x in lines[2*i+2].split(' ')])
        _bookIds = [int(x) for x in lines[2*i+3].split(' ')]
        lib['bookIds'] = np.array(_bookIds)
        tempBookScores = bookScores[(lib['bookIds'],)]
        temp_axis0 = math.ceil(lib['numBooksInLib']/lib['booksSendPerDay'])
        temp_cumsumBookIds = add_zeros(lib['bookIds'][tempBookScores.argsort()[::-1]], (temp_axis0*lib['booksSendPerDay']-lib['numBooksInLib']),begin=False).reshape(temp_axis0, lib['booksSendPerDay'])
        temp_cumsumBookIds = add_zeros(np.cumsum(temp_cumsumBookIds.sum(axis=1)), lib['numDaysReg'])
        if temp_cumsumBookIds.shape[0] > numDays:
            temp_cumsumBookIds = temp_cumsumBookIds[:numDays]
        lib['cumsumBookIds'] = add_zeros(temp_cumsumBookIds, numDays-temp_cumsumBookIds.shape[0], begin=False, val=temp_cumsumBookIds[:-1])
        libs.append(lib)
    #
    while i < numDays:
        libMaxIndex = sorted(libs, key=lambda x: x['cumsumBookIds'][numDays - i])[0].index()
        libMaxInfo = {}
        libMaxInfo['ID'] = libMaxIndex
        tempBookScores = bookScores[(lib['bookIds'],)]
        libMaxBooksTemp = libs[libMaxIndex]['bookIds'][tempBookScores.argsort()[::-1]]
        minInd = min(libMaxBooksTemp.shape[0], (numDays-i-libs[libMaxIndex]['numDaysReg'])*libs[libMaxIndex]['booksSendPerDay'])
        libMaxInfo['Books'] = tempBookScores[:minInd]
        sequence.append(libMaxInfo)
        i += libs[libMaxIndex]['numDaysReg']
        

In [ ]:
filename = "a_example.txt"
with open(filename) as f:
    lines = f.readlines()
    numBooks, numLib, numDays = [int(x) for x in lines[0].split(' ')]
    bookScores = [int(x) for x in lines[1].split(' ')]
    bookScores = np.array(bookScores)
    libs = []
    sequence = []
    for i in range(numLib):
        # numBooksInLib, numDaysReg, booksSendPerDay
        lib = {}
        lib['numBooksInLib'], lib['numDaysReg'], lib['booksSendPerDay'] = tuple([int(x) for x in lines[2*i+2].split(' ')])
        lib['bookIds'] = [int(x) for x in lines[2*i+3].split(' ')]
    
    sortedLibs = sorted(libs, key=lambda x: x['numDaysReg'], reverse=True)
    sequence = []
    j = 0
    i = 0
    while i < numDays:
        sequence.append((sortedLibs[j].index(), sortedLibs[j]['bookIds']))
        j += 1
        i += sortedLibs[j]['numDaysReg']
    with open("output.txt", "w") as f:
        f.write(len(sequence))
        for i in 